# Naive Bayes, Decision Trees with ensemble methods
## CSCI 4622 - Fall 2021
***
**Name**: $Luis R. Corzo$ 
***

## Overview 

Your task for this homework is to build a naive Bayes and a decision tree classifiers in the first 2 problems.
The last problem is about ensemble methods using scikit-learn decision tree as a weak learner.
We'll explore bagging and Random Forest.

In [1]:
import numpy as np
import matplotlib.pylab as plt
import pickle
from sklearn.metrics import precision_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split
%matplotlib inline 

### Problem 1 - Naive Bayes [25 points]
***
Consider the problem of predicting whether a person has a college degree based on age, salary, and Colorado residency.
The dataset looks like the following.

|Age|Salary|Colorado Residency| College degree|
|:------:|:-----------:| :----------:|--:|
| 27 | 41,000 | Yes | Yes |
| 61 | 52,000 | No | No |
| 23 | 24,000 | Yes | No |
| 29 | 77,000 | Yes | Yes |
| 32 | 48,000 | No | Yes |
| 57 | 120,000 | Yes | Yes |
| 22 | 38,000 | Yes | Yes |
| 41 | 45,000 | Yes | No |
| 53 | 26,000 | No | No |
| 48 | 65,000 | Yes | Yes |


In [2]:
features = np.array([[27 , 41000 , 1],
              [61 , 52000 , 0],
              [23 , 24000 , 1],
              [29 , 77000 , 1],
              [32 , 48000 , 0],
              [57 , 120000 , 1],
              [22 , 38000 , 1],
              [41 , 45000 , 1],
              [53 , 26000 , 0],
              [48 , 65000 , 1]])

labels = np.array([1, 0, 0, 1, 1, 1, 1, 0, 0, 1])


1.1 What is our expected accuracy for the baseline case where we predict one label for all rows? (*2 points*)

#BEGIN Workspace 1.1

    I will be using the zero rule algorithm to predict the accuracy for the baseline case. Out of the ten samples in our data set, six have the label "yes" and four have the label "no". (-six samples have a college degree and four do not.) According to the ZeroR benchmark procedure, our expected baseline accuracy is 60%. If we predicted all rows to have "yes" as the label, our accuracy would be of 60%.

#END Workspace 1.1

First, we have to find a way to deal with the continuous features. For now, let's put them into binary bins based on threshold arguments to our classifier - so we can treat this as a tuning parameter.

1.2 Complete `threshold_features` to convert age and salary features to binary ones using the threshold arguments. (*3 points*)

In [3]:
def threshold_features(features, age_threshold, salary_threshold):
    binary_X = features * 1 #This row just creates a "hard copy" of the X array so we can manipulate it as needed

    #BEGIN Workspace 1.2
    #TODO: Threshold the corresponding features

    for sample in binary_X: 
        
        if sample[0] < age_threshold:
            sample[0] = 0
        else:
            sample[0] = 1
            
        if sample[1] < salary_threshold:
            sample[1] = 0
        else:
            sample[1] = 1
        
    
    #END Workspace 1.2

    return binary_X


As seen during the class, given a row $(x_1, x_2, x_3)$, the naive Bayes classifier should assign the label $y$ that
maximizes:

$$\begin{align}
\log [p(y) \prod_i p(x_i | y)] = \log p(y) + \sum_{i} \log p(x_i | y)
\end{align}$$

$p(y)$ and $p(x_i | y)$ are computed using the training set (during `fit` call).

We have defined $p(x_i | y)$ as :
\begin{align}
p(x_i | y) = \frac{N_{y,i}}{N_y}
\end{align}
where $N_{y,i}$ is the number of rows where $y$ and $x_i$ occur together, and $N_y = \sum_i N_{y,i}$.

1.3 Complete the `fit` call by computing the counts and joint counts. Hint: Use `features_counts` to store the contingency
table $N_{y,i}$ for each feature $i$ and then use them to compute $\log p(x_i | y)$ (*10 points*)

1.4 Complete the `predict` call (*5 points*)

In [4]:
class NaiveBayes(object):
    """
    NaiveBayes classifier for binary features and binary labels
    """
    
    def __init__(self, alpha=0.0):
        self.alpha = alpha
        self.classes_counts = None
        self.features_counts = []
        self.classes_log_probabilities = None
        self.features_log_probabilities = [] # same structure as features_count

    def fit(self, X, y):
        
        """
        Parameters
        ----------
        X: binary np.array of shape (n_samples, n_features)
        y: corresponding labels of shape (n_samples,)
        Returns
        -------
        Trained classifier
        """
        
        #BEGIN Workspace 1.3
        #TODO: Compute the counts and joint counts
        
        yes_counts = 0
        no_counts = 0
        
        for label in y:
            
            if label == 1:
                yes_counts += 1
            if label == 0:
                no_counts += 1
        
        counts = np.array([yes_counts, no_counts])
        
        self.classes_counts = counts
        
        log_p_y = np.array([np.log((yes_counts+1)/(len(y)+2)), np.log((no_counts+1)/(len(y)+2))])
        
        self.classes_log_probabilities = log_p_y
    
        log_p_x1 = 0
        log_p_x2 = 0
        log_p_x3 = 0
        
        # joint counts stored are as of the form N_y_xi = [N_yesi, N_noi], 
        # where N_yesi = [(# of counts where xi=1 and y=yes), (# of counts where xi=0 and y=yes)],
        # and N_noi = [(# of counts where xi=1 and y=no), (# of counts where xi=0 and y=no)]
        
        N_yes1 = np.array([0,0]) 
        N_yes2 = np.array([0,0])
        N_yes3 = np.array([0,0])
        
        N_no1 = np.array([0,0])
        N_no2 = np.array([0,0])
        N_no3 = np.array([0,0])
        
        for (row, label) in zip(X, y):
            
            #x1 feature of sample
            if row[0] == 1:       #x1 feature is above threshold 
                
                if label == 1: 
                    N_yes1[0] += 1
            
                if label == 0:
                    N_no1[0] += 1
            
            if row[0] == 0:       #x1 feature is under threshold 
                
                if label == 1:
                    N_yes1[1] += 1
            
                if label == 0:
                    N_no1[1] += 1
            
            
            #x2 feature of sample
            if row[1] == 1:
                
                if label == 1:
                    N_yes2[0] += 1
            
                if label == 0:
                    N_no2[0] += 1
            
            if row[1] == 0:
                
                if label == 1:
                    N_yes2[1] += 1
            
                if label == 0:
                    N_no2[1] += 1
            
            
            #x3 feature of sample
            if row[2] == 1:
                
                if label == 1:
                    N_yes3[0] += 1
            
                if label == 0:
                    N_no3[0] += 1
                
            if row[2] == 0:
                
                if label == 1:
                    N_yes3[1] += 1
            
                if label == 0:
                    N_no3[1] += 1
                    
        N_y_x1 = np.array([N_yes1, N_no1])
        N_y_x2 = np.array([N_yes2, N_no2])
        N_y_x3 = np.array([N_yes3, N_no3])
        
        N_yes = np.array([ [N_yes1[0] + N_yes2[0] + N_yes3[0]], [N_yes1[1] + N_yes2[1] + N_yes3[1]] ])
        N_no = np.array([ [N_no1[0] + N_no2[0] + N_no3[0]], [N_no1[1] + N_no2[1] + N_no3[1]] ])
        
        N_y_xi = np.array([N_y_x1, N_y_x2, N_y_x3])
        
        self.features_counts = N_y_xi
        
        
        log_p_x1_yes = np.array([np.log((N_yes1[0]+self.alpha)/(float(N_yes[0]) + 2*self.alpha)), np.log((N_yes1[1]+self.alpha)/(float(N_yes[1]) + 2*self.alpha))])
        log_p_x2_yes = np.array([np.log((N_yes2[0]+self.alpha)/(float(N_yes[0]) + 2*self.alpha)), np.log((N_yes2[1]+self.alpha)/(float(N_yes[1]) + 2*self.alpha))])
        log_p_x3_yes = np.array([np.log((N_yes3[0]+self.alpha)/(float(N_yes[0]) + 2*self.alpha)), np.log((N_yes3[1]+self.alpha)/(float(N_yes[1]) + 2*self.alpha))])
        
        log_p_x1_no = np.array([np.log((N_no1[0]+self.alpha)/(float(N_no[0]) + 2*self.alpha)), np.log((N_no1[1]+self.alpha)/(float(N_no[1]) + 2*self.alpha))])
        log_p_x2_no = np.array([np.log((N_no2[0]+self.alpha)/(float(N_no[0]) + 2*self.alpha)), np.log((N_no2[1]+self.alpha)/(float(N_no[1]) + 2*self.alpha))])
        log_p_x3_no = np.array([np.log((N_no3[0]+self.alpha)/(float(N_no[0]) + 2*self.alpha)), np.log((N_no3[1]+self.alpha)/(float(N_no[1]) + 2*self.alpha))])
        
        log_p_xi = np.array([log_p_x1_yes, log_p_x2_yes, log_p_x3_yes, log_p_x1_no, log_p_x2_no, log_p_x3_no])
        
        self.features_log_probabilities = log_p_xi
        
        #END Workspace 1.3

        return self

    def predict(self, x_test):
        
        joint_log_likelihood = np.zeros((x_test.shape[0], self.classes_counts.shape[0]))
        y_hat = []
        #BEGIN Workspace 1.4
        #TODO: Find the corresponding labels using Naive bayes logic
        
        for sample in x_test:
            
            x1 = sample[0]
            x2 = sample[1]
            x3 = sample[2]

            log_p_yes = self.classes_log_probabilities[0]

            log_p_x1_yes = self.features_log_probabilities[0][1-x1]

            log_p_x2_yes = self.features_log_probabilities[1][1-x2]

            log_p_x3_yes = self.features_log_probabilities[2][1-x3]

            p_yes_X = log_p_yes + log_p_x1_yes + log_p_x2_yes + log_p_x3_yes


            log_p_no = self.classes_log_probabilities[1]

            log_p_x1_no = self.features_log_probabilities[3][1-x1]

            log_p_x2_no = self.features_log_probabilities[4][1-x2]

            log_p_x3_no = self.features_log_probabilities[5][1-x3]

            p_no_X = log_p_no + log_p_x1_no + log_p_x2_no + log_p_x3_no

            if p_no_X < p_yes_X:
                y_hat.append(1)

            else:
                y_hat.append(0)
        
        #END Workspace 1.4
        return y_hat


1.5 Using age 30 and salary 40,000 as thresholds, transform the features and evaluate (accuracy) the NaiveBayes classifier
on the training data. (*5 points*)

In [5]:
clf = NaiveBayes(alpha=1)
#BEGIN Workspace 1.5
#TODO: Transform features to binary features, fit the classifier, report the accuracy

bin_features = threshold_features(features, 30, 40000)

#x_train, x_test, y_train, y_test = train_test_split(bin_features, labels, test_size=0.2)

print("Actual labels:   ", list(labels))

clf.fit(bin_features, labels)
pred_y = clf.predict(bin_features)

print("Predicted labels:", pred_y)
hit, miss = 0, 0

for (pred_label, label) in zip(pred_y, labels):
    
    if pred_label == label:
        hit += 1
    else:
        miss += 1

accuracy = hit/(hit+miss)    
print("Accuracy:", accuracy)

#END Workspace 1.5

Actual labels:    [1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
Predicted labels: [1, 0, 1, 1, 0, 1, 1, 1, 0, 1]
Accuracy: 0.7


**Bonus question** 1.6 Use the attribute `alpha` of the NaiveBayes to convert it to the smoothed NaiveBayes presented during the class. (*5 points*)

### Problem 2 - Decision trees [25 points]
***
The goal of this problem is to implement the core elements of the Decision Tree classifier.
We do not expect a highly efficient implementation of the functions since the ensemble methods will use the implementation
from scikit-learn.

We start by considering the variable *Colorado residency*.

The leaf nodes of a decision tree act in the same way as in question (1.1) where no feature is used.

2.1 Complete `get_error_in_leaf` to return the count of misclassified instances. (*3 points*)

In [8]:
def get_error_in_leaf(y, indices):
    """
    :param y: all labels
    :param indices: the subset of indexes in the leaf node
    :return: Returns the number of errors in a leaf node of a decision tree.
    """
    
    error_count = 0
    #BEGIN Workspace 2.1
    #TODO: Compute the number of errors in the leaf node (no feature is used)
    
    leaf = np.array([y[i] for i in indices])
    
    ones = np.count_nonzero(leaf)
    zeros = len(leaf) - ones
    
    if ones >= zeros:
        error_count = zeros
    else:
        error_count = ones
    
    #END Workspace 2.1
    return error_count


def value_split_binary_feature(x, y, feature_index, root, criteria_func):  #splits parent node (root) and returns criteria value (information gain)
    """Will be used later to evaluate the criteria gain"""
    left_child = [i for i in root if x[i, feature_index] == 0]
    right_child = [i for i in root if x[i, feature_index] == 1]
    
    return criteria_func(y, root, left_child, right_child)

We will use information gain criteria to decide how to split the root node of our decision tree.

2.2 Complete the `entropy` function. (*5 points*)

2.3 Complete the `information_gain_criteria` to compute the information gained by splitting the root node.
 Print the gain value for splitting based on *Colorado residency* (*5 points*)


In [5]:
def entropy(y, indices):
    """
    :param y: all labels
    :param indices: the indices of data points
    :return: Returns the entropy in the labels for the data points in indices.
    """
    
    entropy_value = 0
    if len(indices) == 0: # deal with corner case when there is no data point.
        return entropy_value
    else:
        #BEGIN Workspace 2.2
        #TODO: Compute the entropy of the labels from indices
        
        error_count = get_error_in_leaf(y, indices)
        non_error_count = len(indices) - error_count
        
        p_1 = (error_count/len(indices))
        p_2 = (non_error_count/len(indices))
        
        if p_1 == 0.0 or p_2 == 0.0:
            entropy == 0
        else:
            entropy_value = (p_1*(np.log2(1/p_1))) + (p_2*(np.log2(1/p_2)))
        
        #END Workspace 2.2
    return entropy_value

def information_gain_criteria(y, root, left_child, right_child):
    """
    :param y: all labels
    :param root: indices of all the data points in the root
    :param left_child: the subset of indices in the left child
    :param right_child: the subset of indices in the right child
    :return: information gain of the split
    """
    information_gain = 0
    #BEGIN Workspace 2.3.a
    #TODO: Compute the information gain of the split
    
    Entropy_prior = entropy(y, root)
    Entropy_left = entropy(y, left_child)
    Entropy_right = entropy(y, right_child)
    
    
    left = np.array([y[i] for i in left_child])
    right = np.array([y[i] for i in right_child])
    
    Entropy_split = ((len(left_child)/len(root))*Entropy_left)+((len(right_child)/len(root))*Entropy_right)
    
    information_gain = Entropy_prior - Entropy_split
    
    
    #END Workspace 2.3.a
    return information_gain

In [10]:
feature_id = 2
info_gain = 0
#BEGIN Workspace 2.3.b
#TODO: report the information gain of the split based on Colorado Residency
x = features
y = labels
root = [i for i in range(len(y))]

info_gain = value_split_binary_feature(x, y, feature_id, root, criteria_func=information_gain_criteria)

print("Information gain of split based on colorado residency: ", "{:.3f}".format(info_gain))
#END Workspace 2.3.b

Information gain of split based on colorado residency:  0.091


Now we have to deal with continuous features for the decision tree.
One way to deal with continuous (or ordinal) data is to define binary features based on thresholding as we've done
for NaiveBayes. But we have to find the optimal threshold based on the criteria we're using.

2.4 Complete the `value_split_continuous_feature` by trying different possible threshold values of feature
of index `feature_index` and return the best criteria value and threshold. (*5 points*)

In [6]:
def value_split_continuous_feature(x, y, feature_index, root, criteria_func=information_gain_criteria):
    """
    :param x: all feature values
    :param y: all labels
    :param feature_index: feature id to split the tree based on
    :param root: indexes of all the data points in the root
    :param criteria_func: the splitting criteria function
    :return: Return the best value and its corresponding threshold by splitting based on a continuous feature.
    """

    best_value, best_thres = 0, 0
    
    #BEGIN Workspace 2.4
    #TODO: Complete the function as detailed in the question and function description
    
    values = []
    thresholds = []
    
    if feature_index == 0:
        
        salary_threshold = 53600
        
        for i in range(1,12):
            inc = i*3
            age_threshold = 26 + inc
            bin_x = threshold_features(x, age_threshold, salary_threshold)
            
            values.append(value_split_binary_feature(bin_x, y, feature_index, root, criteria_func))
            thresholds.append(age_threshold)
            
    if feature_index == 1:
        
        age_threshold = 38
        
        for i in range(1,30):
            inc = 3000*i
            salary_threshold = 26000 + inc
            bin_x = threshold_features(x, age_threshold, salary_threshold)
            
            values.append(value_split_binary_feature(bin_x, y, feature_index, root, criteria_func))
            thresholds.append(salary_threshold)
    
    sorted_vals = np.sort(values)
    best_value = sorted_vals[len(values)-1]
    best_value_idx = values.index(best_value)
    best_thres = thresholds[best_value_idx]
    #END Workspace 2.4
    return best_value, best_thres

2.5 Find the best thresholds for age and salary. Print their corresponding information gains. (*5 points*)

In [9]:
x = features
y = labels
root = list(range(len(labels))) # root includes all data points
#BEGIN Workspace 2.5
#TODO: Report the best thresholds for age and salary and their split information gains

age_val_split = value_split_continuous_feature(x, y, 0, root, criteria_func=information_gain_criteria)
salary_val_split = value_split_continuous_feature(x, y, 1, root, criteria_func=information_gain_criteria)

print("Best age threshold: ", age_val_split[1])
print("Split information gain (best age threshold): ", age_val_split[0])
print("\n")
print("Best salary threshold: ", salary_val_split[1])
print("Split information gain (best salary threshold): ", salary_val_split[0])
print("\n")
#END Workspace 2.5

Best age threshold:  59
Split information gain (best age threshold):  0.14448434380562825


Best salary threshold:  29000
Split information gain (best salary threshold):  0.3219280948873624




2.6 Based on the obtained information gains, if we build a decision stump (decision tree with depth 1) greedily,
which feature should we choose? Why? What's the resulting accuracy? (*2 points*)

#BEGIN Workspace 2.6.a

- Which feature should we pick for the decision stump? Why?

    Taking a greedy approach means that we should take the optimal local choice at each node-split. Dealing with building a decision stump is can be thought of as dealing with a single decision split on the root node. Given our greedy approach, we should chose the split based on the feature which maximizes our information gain for such split. 

 The information gains for each feature-split are the following:

    a) Age (-with threshold value of 59): 0.144 
    b) Salary (-with threshold value of 29000): 0.321
    c) Colorado residency:  0.091
    
 Concerning only the initial root-split, choosing the Salary feature (-threshold of 29000) is the optimal local choice.
 
#END Workspace 2.6.a

In [15]:
#BEGIN Workspace 2.6.b
#TODO: Split based on the chosen feature and compute the accuracy (use get_error_in_leaf)

def split_and_accuracy(x, y, age_threshold, salary_threshold, root):

    bin_x = threshold_features(x, age_threshold, salary_threshold)
    
    left_child = [i for i in root if bin_x[i, 1] == 0]
    right_child = [i for i in root if bin_x[i, 1] == 1]
    
    parent = np.array([y[i] for i in root])
    left = np.array([y[i] for i in left_child])
    right = np.array([y[i] for i in right_child])
    
    print("\n")
    print("Parent:", parent)
    print("Left child:", left, ",  Right child:", right)
    print("\n")
    
    left_errors = get_error_in_leaf(y, left_child)
    right_errors = get_error_in_leaf(y, right_child)
    
    print("Left child errors: ", left_errors)
    print("Right child errors: ", right_errors)
    
    hits = (len(left_child)-left_errors) + (len(right_child)-right_errors)
    accuracy = hits/len(root)
    
    print("\n")
    print("Accuracy: ", accuracy)
    print("\n")

age_threshold = 59
salary_threshold = 29000

split_and_accuracy(x, y, age_threshold, salary_threshold, root)
#BEGIN Workspace 2.6.b



Parent: [1 0 0 1 1 1 1 0 0 1]
Left child: [0 0] ,  Right child: [1 0 1 1 1 1 0 1]


Left child errors:  0
Right child errors:  2


Accuracy:  0.8




**Bonus Question**

2.7 You now have all the ingredients to build a decision tree recursively.
You can build a decision tree of depth two and report its classification error on the training data and the tree.(*5 points*)

In [94]:
#BEGIN Workspace 2.7
#TODO: Build a Decision Tree of Depth 2 using age, salary and the previously computed thresholds

def buildTree(x, y, depth, root, errors):
    
    left_child = [i for i in root if x[i][depth-1] == 0]
    right_child = [i for i in root if x[i][depth-1] == 1]

    left = [y[i] for i in left_child]
    right = [y[i] for i in right_child]

    left_errors = get_error_in_leaf(y, left_child)
    right_errors = get_error_in_leaf(y, right_child)
   
    if depth != 0:
        
        depth -= 1
        
        parent = [y[i] for i in root]
        print("Root", parent)
        
        if depth != 0:
            
            if left_errors == 0 and right_errors != 0:
                print("Left leaf",left,",  Right child", right)
                
            if left_errors != 0 and right_errors == 0:
                print("Left child",left,",  Right leaf", right)
        else:
            print("Left leaf",left,",  Right leaf", right)

        if left_errors != 0:
            print("\n")
            return buildTree(x,y,depth,left_child, left_errors)

        if right_errors != 0:
            print("\n")
            return buildTree(x,y,depth,right_child, right_errors)
            
    else:
        return errors
        
        
    
bin_x = threshold_features(x, age_threshold, salary_threshold)
new_bin_x = []


for row in bin_x:
    row = list(row)
    del row[2]
    new_bin_x.append(row)
    
    


errors = 0
errors = buildTree(new_bin_x, y,2, root, errors)
print("Errors in leaves:", errors)

accuracy = (len(y) - errors)/len(y)
print("Accuracy: ",accuracy)
#END Workspace 2.7

Root [1, 0, 0, 1, 1, 1, 1, 0, 0, 1]
Left leaf [0, 0] ,  Right child [1, 0, 1, 1, 1, 1, 0, 1]


Root [1, 0, 1, 1, 1, 1, 0, 1]
Left leaf [1, 1, 1, 1, 1, 0, 1] ,  Right leaf [0]


Errors in leaves: 1
Accuracy:  0.9


Problem 3  - Decision Tree Ensembles: Bagging and (BONUS: Boosting) [50 points]
---

We are going to predict house price levels using decision tree ensembles.

In this classification problem, we compare Decision trees and it's ensembles - Bagging and Boosting on House Price prediction [dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

Our *weak learner* for this problem is the DecisionTreeClassifier from scikit-learn with `max_depth=10`.

We start first by loading preprocessed data that we'll use. Since the original data is for regression, we have first to transform
`y_train` and `y_test` to discrete values reflecting price level.

|Price range| Label|
|:----------:|--:|
| $ P < $125000|0|
|125000$\leq P < $ 160000| 1 |
|160000$ \leq P < $ 200000| 2 |
|200000$ \leq P $ | 3 |

3.1 Start by transforming`y_train` and `y_test` to discrete values using the provided ranges. (*3 points*)

In [12]:
X_train, X_test, y_train, y_test = pickle.load(open('./data/test_train.pkl','rb'))
#BEGIN Workspace 3.1
#TODO: Discretize y_train and y_test

#print(y_train)
#print(y_test)

def discretize(data):
    
    discrete_data = []
    
    for sample in data:
        
        if sample < 125000:
            discrete_data.append(0)
            
        if sample >= 125000 and sample < 160000:
            discrete_data.append(1)
            
        if sample >= 160000 and sample < 200000:
            discrete_data.append(2)
            
        if sample >=  200000:
            discrete_data.append(3)
    
    return discrete_data

y_train = discretize(y_train)
y_test = discretize(y_test)

#END Workspace 3.1
print(np.unique(y_train), X_train.shape)
print(np.unique(y_test), X_test.shape)

[0 1 2 3] (1166, 79)
[0 1 2 3] (292, 79)


3.2 Complete the `ensemble_test` class to `fit` the model received as parameter and store the metrics and running time. (*5 points*)

3.3 Complete `plot_metric` to show and compare different statistics of each model in a bar chart. (*5 points*)

Later we will also use `ensemble_test` class to plot score, metric and time taken to fit the data.

In [13]:
def get_weak_learner():
    """Return a new instance of out chosen weak learner"""
    return DecisionTreeClassifier(max_depth=10)

class EnsembleTest:
    """
        Test multiple model performance
    """

    def __init__(self, x_train, y_train, x_test, y_test):
        """
        initialize data partitions
        """
        self.scores = {}
        self.execution_time = {}
        self.metric = {}
        self.x_train = X_train
        self.y_train = y_train
        self.x_test = X_test
        self.y_test = y_test
        self.score_name ='Mean accuracy'
        self.metric_name = 'Precision(micro)'

    def fit_model(self, model, name):
        """
        Fit the model on train data.
        predict on test and store score and execution time for each fit.
        :param model: model
        :param name: name of model
        """
        start = time()
        #BEGIN Workspace 3.2
        #TODO: Fit the model and get the predictions
        #       train and test data are treated as global variables
        #Hint: self.scores[name] = precision_score(?, average="micro") # in multi-class, micro implies treating it as binary precision
        #Hint self.metric[name] = Accuracy
        
        
        model.fit(self.x_train, self.y_train)
        
        y_pred = model.predict(self.x_test)
        
        hit, miss = 0, 0
        #print(len(y_pred),len(self.y_test))

        for (pred_label, label) in zip(y_pred, self.y_test):

            if pred_label == label:
                hit += 1
            else:
                miss += 1

        Accuracy = hit/(hit+miss)  

        self.scores[name] = precision_score(self.y_test, y_pred, average="micro")
        self.metric[name] = Accuracy
        
        
        #END Workspace 3.2
        self.execution_time[name] = time() - start

    def print_result(self):
        """
            print results for all models trained and tested.
        """
        models_cross = pd.DataFrame({
            'Model'         : list(self.metric.keys()),
             self.score_name     : list(self.scores.values()),
             self.metric_name    : list(self.metric.values()),
            'Execution time': list(self.execution_time.values())})
        print(models_cross.sort_values(by=self.score_name, ascending=False))

    def plot_metric(self):
        #BEGIN Workspace 3.3
        #TODO: plot bar chart for each metric : time, metric, score
        times = []
        metrics = []
        scores =[]
        
        for time in self.execution_time.values():
            times.append(time)
            
        for metric in self.metric.values():
            metrics.append(metric)
        
        for score in self.scores.values():
            scores.append(score)
        
        plt.bar(range(len(times)),times, color='blue')
        plt.show()
        
        plt.bar(range(len(metrics)),metrics, color='purple')
        plt.show()
        
        plt.bar(range(len(scores)),scores, color='green')
        plt.show()
            
        #END Workspace 3.3

3.4 Test `EnsembleTest` using our weak learner returned by `get_weak_learner` (*2 points*)

In [14]:
# create a handler for ensemble_test, use the created handler for fitting different models.
ensemble_handler = EnsembleTest(X_train,y_train,X_test,y_test)
#BEGIN Workspace 3.4
#TODO: Initialize weak learner and fit ensemble_handler
model = get_weak_learner()

ensemble_handler.fit_model(model, "DecisionTreeTest")

#END Workspace 3.4
ensemble_handler.print_result()

              Model  Mean accuracy  Precision(micro)  Execution time
0  DecisionTreeTest       0.681507          0.681507        0.021736


**Bagging:**

Bagging consists of training a set of weak learners using random subsets of the train data.

3.5 First, complete `sample_data` to return a random sample of size `sample_ratio * len(X_train)` of features and labels (*2 points*)

3.6 Complete `fit` by building `n_estimators` of DecisionTreeClassifier, each trained on random sample of the data (*5 points*)

3.7 Complete `predict` method to return the most likely label by combining different estimators predictions. 
Use a simple majority / plurality vote system similar to the one used in your KNNClassifier in Problem Set 1. However, in this case, to break a tie you should use `predict_log_proba` or `predict_proba` method of DecisionTreeClassifier:
[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.predict_proba) (*2 points*)

In [22]:
class BaggingEnsemble(object):

    def __init__(self, n_estimators, sample_ratio):
        self.n_estimators = n_estimators
        self.sample_ratio = sample_ratio
        self.estimators = []
        

    def sample_data(self, x_train, y_train):
        x_sample, y_sample = None, None
        #BEGIN Workspace 3.5
        #TODO: sample random subset of size sample_ratio * len(X_train)
        
        size = int(np.floor(self.sample_ratio*len(x_train)))
        
        sample_idx = np.random.choice(len(x_train), size, replace=True)
        
        x_sample = [x_train[i] for i in sample_idx]
        y_sample = [y_train[i] for i in sample_idx]
        
        
        #END Workspace 3.5
        return x_sample, y_sample

    def fit(self, x_train, y_train):
        
        for _ in range(self.n_estimators):
            #BEGIN Workspace 3.6
            #TODO: sample data and create new weak learned trained on the sample
            
            x_sample, y_sample  = self.sample_data(x_train, y_train)
            
            weak_learner = get_weak_learner()
            weak_learner.fit(x_sample, y_sample)
            
            self.estimators.append(weak_learner)
            
            
            #END Workspace 3.6

    def predict(self, X_test):
        predicted_proba = 0
        answer = 0
        #BEGIN Workspace 3.7
        #TODO: go through the trained estimators and acummualte their predicted_proba to get the mostly likely label
        
        estimators_labels = []
        
        for estimator in self.estimators:
            estimators_labels.append(estimator.predict_proba(X_test))
            
        
        acc_probs = estimators_labels[0]
    
        for i in range(1, self.n_estimators):
            acc_probs += estimators_labels[i]
        
        likely_labels = []
        
        for row in acc_probs:
            max_p = max(row)
            label = np.where(row==max_p)
            likely_labels.append(label[0].item(0))
            
        answer = likely_labels    
        #END Workspace 3.7
        return answer


In [23]:
# This cell should run without errors
ensemble_handler.fit_model(BaggingEnsemble(10, 0.9),'Bagging')
ensemble_handler.print_result()


              Model  Mean accuracy  Precision(micro)  Execution time
1           Bagging       0.739726          0.739726        0.158844
0  DecisionTreeTest       0.681507          0.681507        0.021736


**Random Forest**
Random Forest has an additional layer of randomness compared to Bagging: we also sample a random subset of the features.

3.8 First, complete `sample_data` to return a random sample of size `sample_ratio * len(X_train)` of labels and `feature_ratio * num_features` of features (*2 points*)

3.9 Complete `fit` by building `n_estimators` of DecisionTreeClassifier, each trained on random sample of the data.
Make sure to keep track of the sampled features for each estimator to use them in the prediction step. (*5 points*)

3.10 Complete `predict` method to return the most likely label by combining different estimators predictions.
Use a simple majority / plurality vote system similar to the one used in your KNNClassifier in Problem Set 1. However, in this case, to break a tie you should use `predict_log_proba` or `predict_proba`  method of DecisionTreeClassifier:
[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.predict_proba) (*3 points*)


In [43]:
class RandomForest(object):

    def __init__(self, n_estimators, sample_ratio, features_ratio):
        self.n_estimators = n_estimators
        self.sample_ratio = sample_ratio
        self.features_ratio = features_ratio
        self.estimators = []
        self.features_indices = []

    def sample_data(self, x_train, y_train):
        X_sample, y_sample, features_indices = None, None, None
        #BEGIN Workspace 3.8
        #TODO: sample random subset of size sample_ratio * len(X_train) and subset of features of size
        #         features_ratio * num_features
        
        samples_size = int(np.floor(self.sample_ratio*len(x_train)))
        features_size = int(np.floor(self.features_ratio*len(x_train[0])))
        
        samples_idx = np.random.choice(len(x_train), samples_size)
        features_indices = np.random.choice(len(x_train[0]), features_size)
        
        x_sample = [x_train[i] for i in samples_idx]
        y_sample = [y_train[i] for i in samples_idx]
        

        #END Workspace 3.8
        return x_sample, y_sample, features_indices

    def fit(self, x_train, y_train):
        for _ in range(self.n_estimators):
            #BEGIN Workspace 3.9
            #TODO: sample data with random subset of rows and features using sample_data
            #Hint: keep track of the features indices in features_indices to use in predict
            
            x_samples, y_samples, features_idx  = self.sample_data(x_train, y_train)
            
            self.features_indices.append(features_idx)
            
            x_samples_rforest = []
            
            for sample in x_samples:
                
                x_smpl = [sample[i] for i in features_idx]
                x_samples_rforest.append(x_smpl)
            
            weak_learner = get_weak_learner()
            weak_learner.fit(x_samples_rforest, y_samples)
            
            self.estimators.append(weak_learner)
            
            #END Workspace 3.9

    def predict(self, X_test):
        predicted_proba = 0
        answer = 0
        #BEGIN Workspace 3.10
        #TODO: compute cumulative sum of predict proba from estimators and return the labels with highest likelihood
        
        
        estimators_labels = []
        inc = 0
        
        for estimator in self.estimators:
            x_test_rforest = X_test[:,self.features_indices[inc]]
            estimators_labels.append(estimator.predict_proba(x_test_rforest))
            inc+=1
        
        acc_probs = estimators_labels[0]
    
        for i in range(1, self.n_estimators):
            acc_probs += estimators_labels[i]
        
        likely_labels = []
        
        for row in acc_probs:
            max_p = max(row)
            label = np.where(row==max_p)
            likely_labels.append(label[0].item(0))
            
        answer = likely_labels 
        
        #END Workspace 3.10
        return answer

In [45]:
# This cell should run without errors
ensemble_handler.fit_model(RandomForest(20, sample_ratio=0.9, features_ratio=0.8), 'RandomForest')
ensemble_handler.print_result()

              Model  Mean accuracy  Precision(micro)  Execution time
2      RandomForest       0.770548          0.770548        0.695282
1           Bagging       0.739726          0.739726        0.158844
0  DecisionTreeTest       0.681507          0.681507        0.021736
